<h3><span style="color:red"> Note: <b>%%bash</b> blocks does not show intermediate outputs, you may want to copy and run these in terminal </span></h3>

# Prepare Data

## Download

Go to http://www.cvlibs.net/datasets/kitti/eval_object.php and get URL to download these files:
* Left color images of object data set | `data_object_image_2.zip` | 12GB | [link](http://www.cvlibs.net/download.php?file=data_object_image_2.zip)
* Training labels of object data set | `data_object_label_2.zip` | 5MB | [link](http://www.cvlibs.net/download.php?file=data_object_label_2.zip)

Alternative source https://www.kaggle.com/twaldo/kitti-object-detection

In [ ]:
%%bash
mkdir -p /data/kitti
cd /data/kitti

export IMAGE_URL='https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip' # originally http://kitti.is.tue.mpg.de/kitti/data_object_image_2.zip 
export LABEL_URL='https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip' # http://kitti.is.tue.mpg.de/kitti/data_object_label_2.zip
if [[ -d training/image_2 ]] && [[ $(ls training/image_2/*.png  | wc -l) -ge 7481 ]];then  
  echo "No need to Download"
  echo "Images already there in ${PWD}/training/image_2/"
else
  echo "Starting Download" 
  wget -c --show-progress --progress=bar ${IMAGE_URL}
  wget -c --show-progress --progress=bar ${LABEL_URL}
  echo "Extracting" 
  unzip -q data_object_image_2.zip
  unzip -q data_object_label_2.zip
fi


## Validate

In [ ]:
%%bash

cd /data/kitti

echo -n "Images: " ; ls training/image_2/*.png | wc -l # Should be # 7481

echo "Structure"
tree --filelimit 20 


Expected path
<data_dir>/training/label_2 (annotations) and
<data_dir>/data_object_image_2/training/image_2


## Symlink to match script

`dataset_tools/create_kitti_tf_record.py` needs `data_object_image_2/training` to point to `training`

Setting up relative symlink to `../training`

In [ ]:
%%bash
cd /data/kitti

if [ ! -d "data_object_image_2" ] 
then
    echo "Directory data_object_image_2 DOES NOT exists." 
    mkdir data_object_image_2
    ln -s ../training data_object_image_2/training
fi

ls /data/kitti/data_object_image_2/training/

## Convert to tfrecord

In [ ]:
%%bash
# !ls ${TF_OBJECT_DETECTION_API}/data
mkdir -p /data/kitti/kitti_img2d_tf/
if [[ ! -f "/data/kitti/kitti_img2d_tf/kitti_train.tfrecord" ]]
then
  python ${TF_OBJECT_DETECTION_API}/dataset_tools/create_kitti_tf_record.py \
        --data_dir=/data/kitti \
        --output_path=/data/kitti/kitti_img2d_tf/kitti \
        --label_map_path=${TF_OBJECT_DETECTION_API}/data/kitti_label_map.pbtxt
fi
ls --size --block-size=M /data/kitti/kitti_img2d_tf/

# Prepare Training

## Get pretrained model

In [ ]:
%%bash
export MODEL_NAME='faster_rcnn_resnet101_kitti_2018_01_28'
mkdir -p /data/pretrained_models
cd /data/pretrained_models

if [[ -d /data/pretrained_models/${MODEL_NAME} ]]
then 
  echo "Model ${MODEL_NAME} already exists" 
else
  echo "Downloading model ${MODEL_NAME}" 
  wget -c http://download.tensorflow.org/models/object_detection/${MODEL_NAME}.tar.gz 
  tar xvzf ${MODEL_NAME}.tar.gz
fi

echo "Model path: /data/pretrained_models/${MODEL_NAME}"

## Configure pipeline
Set the paths `PATH_TO_BE_CONFIGURED` to correct values

In [ ]:
%%bash
export MODEL_NAME='faster_rcnn_resnet101_kitti_2018_01_28'
export PRETRAINED_MODEL_PATH="/data/pretrained_models/${MODEL_NAME}"

export MODEL_OUT_PATH="/data/kitti/trained_model"
mkdir -p ${MODEL_OUT_PATH}

#cp ${TF_OBJECT_DETECTION_API}/samples/configs/faster_rcnn_resnet101_kitti.config ${MODEL_OUT_PATH}/kitti_pipeline.config
cp ${PRETRAINED_MODEL_PATH}/pipeline.config ${MODEL_OUT_PATH}/kitti_pipeline.config

sed -i "s|PATH_TO_BE_CONFIGURED/model.ckpt|${PRETRAINED_MODEL_PATH}/model.ckpt|g" ${MODEL_OUT_PATH}/kitti_pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/kitti_label_map.pbtxt|${TF_OBJECT_DETECTION_API}/data/kitti_label_map.pbtxt|g" ${MODEL_OUT_PATH}/kitti_pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/kitti_train.tfrecord|/data/kitti/kitti_img2d_tf/kitti_train.tfrecord|g" ${MODEL_OUT_PATH}/kitti_pipeline.config
sed -i "s|PATH_TO_BE_CONFIGURED/kitti_val.tfrecord|/data/kitti/kitti_img2d_tf/kitti_val.tfrecord|g" ${MODEL_OUT_PATH}/kitti_pipeline.config
sed -i "s|step: 0|step: 1|g" ${MODEL_OUT_PATH}/kitti_pipeline.config # Bug fix
sed -i "s|coco_metrics|coco_detection_metrics|g" ${MODEL_OUT_PATH}/kitti_pipeline.config 

# Check the final config manually
cat ${MODEL_OUT_PATH}/kitti_pipeline.config

# Training
## Setup Tensorboard

In [ ]:
%load_ext tensorboard.notebook
%tensorboard --logdir /data/kitti/trained_model/

#from tensorboard import notebook
#notebook.display(height = 1000)

## Evalute pretrained model

In [ ]:
!python ${TF_OBJECT_DETECTION_API}/model_main.py \
  --pipeline_config_path=/data/kitti/trained_model/kitti_pipeline.config \
  --checkpoint_dir=/data/pretrained_models/faster_rcnn_resnet101_kitti_2018_01_28 \
  --model_dir=/data/kitti/trained_model/faster_rcnn_resnet101_kitti # \
  # --run_once=True 


## Train (fine tune)

In [ ]:
!python ${TF_OBJECT_DETECTION_API}/model_main.py \
    --model_dir=/data/kitti/trained_model/faster_rcnn_resnet101_kitti \
    --pipeline_config_path=/data/kitti/trained_model/kitti_pipeline.config \
    --alsologtostderr \
    --num_train_step=50000
    

# Legacy
## Eval

In [ ]:
!python ${TF_OBJECT_DETECTION_API}/legacy/eval.py \
    --pipeline_config_path=/data/kitti/trained_model/kitti_pipeline.config \
    --eval_dir=/data/kitti/trained_model/faster_rcnn_resnet101_kitti  \
    --checkpoint_dir=/data/pretrained_models/faster_rcnn_resnet101_kitti_2018_01_28

## Train

In [ ]:
!python ${TF_OBJECT_DETECTION_API}/legacy/train.py \
    --pipeline_config_path=/data/kitti/trained_model/kitti_pipeline.config \
    --train_dir=/data/kitti/trained_model/faster_rcnn_resnet101_kitti  \
    --checkpoint_dir=/data/pretrained_models/faster_rcnn_resnet101_kitti_2018_01_28